In [0]:
select * from puc.gold.fato_solicitacao_credito

In [0]:
select * from puc.gold.fato_risco_cliente_mes

In [0]:
-- Volume concedido por mês (total e quantidade)

SELECT
  ano_mes,
  COUNT(*)                         AS qtd_solicitacoes,
  SUM(valor_emprestimo)            AS vl_emprestimo_total,
  AVG(valor_emprestimo)            AS vl_emprestimo_medio
FROM puc.gold.fato_solicitacao_credito
GROUP BY ano_mes
ORDER BY ano_mes;


In [0]:
SELECT
  dt.ano_mes,
  dp.produto,
  SUM(f.valor_emprestimo) AS volume
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo   dt ON f. = dt.sk_tempo
JOIN puc.gold.dim_produto dp ON f.sk_produto = dp.sk_produto
GROUP BY dt.ano_mes, dp.produto;

In [0]:
SELECT
  dt.ano_mes,
  AVG(f.valor_emprestimo) AS media_valor_emprestimo,
  SUM(f.valor_emprestimo) AS total_valor_emprestimo,
  COUNT(*)                AS qtd_solicitacoes
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo dt
  ON f.dt_solicitacao = dt.dt_solicitacao
GROUP BY dt.ano_mes
ORDER BY dt.ano_mes;

In [0]:
SELECT
  dt.ano_mes,
  AVG(f.valor_emprestimo) AS media_valor_emprestimo,
  SUM(f.valor_emprestimo) AS total_valor_emprestimo,
  COUNT(*)                AS qtd_solicitacoes
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo dt
  ON f.dt_solicitacao = dt.dt_solicitacao
GROUP BY dt.ano_mes
ORDER BY dt.ano_mes;

In [0]:
SELECT
  dt.dia_semana,              -- se existir na sua dim_tempo
  COUNT(*)              AS qtd_solicitacoes,
  SUM(f.valor_emprestimo) AS vl_total,
  AVG(f.valor_emprestimo) AS vl_medio
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo dt
  ON f.dt_solicitacao = dt.dt_solicitacao
GROUP BY dt.dia_semana
ORDER BY dt.dia_semana;

In [0]:
SELECT
  dt.dt_solicitacao,
  dp.produto,
  COUNT(*)                AS qtd_solicitacoes,
  SUM(f.valor_emprestimo) AS vl_total,
  AVG(f.valor_emprestimo) AS vl_medio
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo   dt ON f.dt_solicitacao = dt.dt_solicitacao
JOIN puc.gold.dim_produto dp ON f.sk_produto     = dp.sk_produto
GROUP BY dt.dt_solicitacao, dp.produto
ORDER BY dt.dt_solicitacao, dp.produto;

In [0]:
SELECT
  dt.dt_solicitacao,
  dp.produto,
  COUNT(*)                AS qtd_solicitacoes,
  SUM(f.valor_emprestimo) AS vl_total,
  AVG(f.valor_emprestimo) AS vl_medio
FROM puc.gold.fato_solicitacao_credito f
JOIN puc.gold.dim_tempo   dt ON f.dt_solicitacao = dt.dt_solicitacao
JOIN puc.gold.dim_produto dp ON f.sk_produto     = dp.sk_produto
GROUP BY dt.dt_solicitacao, dp.produto
ORDER BY dt.dt_solicitacao, dp.produto;


In [0]:
WITH base AS (
  SELECT
    dt.ano_mes,
    dp.produto,
    SUM(f.valor_emprestimo) AS vl_total
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo   dt ON f.dt_solicitacao = dt.dt_solicitacao
  JOIN puc.gold.dim_produto dp ON f.sk_produto     = dp.sk_produto
  GROUP BY dt.ano_mes, dp.produto
)
SELECT
  *,
  dense_rank() OVER (PARTITION BY ano_mes ORDER BY vl_total DESC) AS rank_mes
FROM base
ORDER BY ano_mes, rank_mes;


In [0]:
WITH sol AS (
  SELECT
    dt.ano_mes,
    f.sk_cliente,
    f.sk_produto,
    f.sk_canal,
    f.sk_localizacao,
    COUNT(*) AS qtd_solicitacoes,
    SUM(f.valor_emprestimo) AS vl_emprestimo_total,
    AVG(CAST(f.inadimplente AS DOUBLE)) AS tx_inadimplencia
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo dt
    ON f.dt_solicitacao = dt.dt_solicitacao
  GROUP BY
    dt.ano_mes, f.sk_cliente, f.sk_produto, f.sk_canal, f.sk_localizacao
),
ris AS (
  SELECT
    date_format(dt_mes, 'yyyy-MM') AS ano_mes,
    sk_cliente,
    sk_produto,
    sk_canal,
    sk_localizacao,
    SUM(perda_esperada_soma) AS el_total,
    AVG(pd_model_media)      AS pd_model_media,
    SUM(ead_soma)            AS ead_total,
    AVG(lgd_media)           AS lgd_media
  FROM puc.gold.fato_risco_cliente_mes
  GROUP BY
    date_format(dt_mes, 'yyyy-MM'), sk_cliente, sk_produto, sk_canal, sk_localizacao
)
SELECT
  sol.ano_mes,
  dp.produto,
  dc.canal,
  dl.uf,
  dl.regiao,

  SUM(sol.qtd_solicitacoes)      AS qtd_solicitacoes,
  SUM(sol.vl_emprestimo_total)   AS vl_emprestimo_total,
  AVG(sol.tx_inadimplencia)      AS tx_inadimplencia_media,

  SUM(ris.el_total)              AS el_total,
  AVG(ris.pd_model_media)        AS pd_model_media,
  SUM(ris.ead_total)             AS ead_total,
  AVG(ris.lgd_media)             AS lgd_media
FROM sol
JOIN ris
  ON  sol.ano_mes = ris.ano_mes
  AND sol.sk_cliente = ris.sk_cliente
  AND sol.sk_produto = ris.sk_produto
  AND sol.sk_canal = ris.sk_canal
  AND sol.sk_localizacao = ris.sk_localizacao
JOIN puc.gold.dim_produto dp      ON sol.sk_produto = dp.sk_produto
JOIN puc.gold.dim_canal dc        ON sol.sk_canal = dc.sk_canal
JOIN puc.gold.dim_localizacao dl  ON sol.sk_localizacao = dl.sk_localizacao
GROUP BY sol.ano_mes, dp.produto, dc.canal, dl.uf, dl.regiao
ORDER BY sol.ano_mes, vl_emprestimo_total DESC;


In [0]:
WITH sol AS (
  SELECT
    date_format(date_trunc('month', data_solicitacao), 'yyyy-MM') AS ano_mes,
    produto,
    canal,
    uf,
    regiao,
    COUNT(*) AS qtd_solicitacoes,
    SUM(valor_emprestimo) AS vl_emprestimo_total,
    AVG(CAST(inadimplente AS DOUBLE)) AS tx_inadimplencia
  FROM puc.silver.base_pd
  GROUP BY
    date_format(date_trunc('month', data_solicitacao), 'yyyy-MM'),
    produto, canal, uf, regiao
),
ris AS (
  SELECT
    date_format(date_trunc('month', data_solicitacao), 'yyyy-MM') AS ano_mes,
    produto,
    canal,
    uf,
    regiao,
    SUM(pd_model * ead * lgd) AS el_total,
    AVG(pd_model)             AS pd_model_media,
    SUM(ead)                  AS ead_total,
    AVG(lgd)                  AS lgd_media
  FROM puc.silver.base_pd
  GROUP BY
    date_format(date_trunc('month', data_solicitacao), 'yyyy-MM'),
    produto, canal, uf, regiao
)
SELECT
  sol.ano_mes,
  sol.produto,
  sol.canal,
  sol.uf,
  sol.regiao,
  sol.qtd_solicitacoes,
  sol.vl_emprestimo_total,
  sol.tx_inadimplencia,
  ris.el_total,
  ris.pd_model_media,
  ris.ead_total,
  ris.lgd_media
FROM sol
LEFT JOIN ris
  ON  sol.ano_mes = ris.ano_mes
  AND sol.produto = ris.produto
  AND sol.canal   = ris.canal
  AND sol.uf      = ris.uf
  AND sol.regiao  = ris.regiao
ORDER BY sol.ano_mes, sol.vl_emprestimo_total DESC;


In [0]:
-- Percentual entre o total da perda esperada e o total de empréstimos usando granularidade de ano_mes e região

WITH sol AS (
  SELECT
    dt.ano_mes,
    f.sk_localizacao,
    SUM(f.valor_emprestimo) AS vl_emprestimo_total_mes
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo dt
    ON f.dt_solicitacao = dt.dt_solicitacao
  GROUP BY dt.ano_mes, f.sk_localizacao
),
ris AS (
  SELECT
    dt.ano_mes,
    fr.sk_localizacao,
    SUM(fr.perda_esperada_soma) AS perda_esperada_total_mes
  FROM puc.gold.fato_risco_cliente_mes fr
  JOIN puc.gold.dim_tempo dt
    ON fr.dt_mes = dt.dt_solicitacao
  GROUP BY dt.ano_mes, fr.sk_localizacao
)
SELECT
  sol.ano_mes,
  dl.uf,
  dl.regiao,
  sol.vl_emprestimo_total_mes,
  COALESCE(ris.perda_esperada_total_mes, 0) AS perda_esperada_total_mes,

  -- percentual da perda sobre o empréstimo
  ROUND(
    (COALESCE(ris.perda_esperada_total_mes, 0) / NULLIF(sol.vl_emprestimo_total_mes, 0)) * 100,
    3
  ) AS pct_perda_sobre_emprestimo

FROM sol
LEFT JOIN ris
  ON sol.ano_mes = ris.ano_mes
 AND sol.sk_localizacao = ris.sk_localizacao
LEFT JOIN puc.gold.dim_localizacao dl
  ON sol.sk_localizacao = dl.sk_localizacao
ORDER BY sol.ano_mes, dl.regiao, dl.uf;


In [0]:
-- Percentual entre o total da perda esperada e o total de empréstimos usando granularidade de ano e região

WITH sol AS (
  SELECT
    dt.ano,
    f.sk_localizacao,
    SUM(f.valor_emprestimo) AS vl_emprestimo_total_ano
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo dt
    ON f.dt_solicitacao = dt.dt_solicitacao
  GROUP BY dt.ano, f.sk_localizacao
),
ris AS (
  SELECT
    dt.ano,
    fr.sk_localizacao,
    SUM(fr.perda_esperada_soma) AS perda_esperada_total_ano
  FROM puc.gold.fato_risco_cliente_mes fr
  JOIN puc.gold.dim_tempo dt
    ON fr.dt_mes = dt.dt_solicitacao
  GROUP BY dt.ano, fr.sk_localizacao
)
SELECT
  sol.ano,
  dl.uf,
  dl.regiao,
  sol.vl_emprestimo_total_ano,
  COALESCE(ris.perda_esperada_total_ano, 0) AS perda_esperada_total_ano,

  -- percentual da perda sobre o empréstimo (em %)
  ROUND(
    (COALESCE(ris.perda_esperada_total_ano, 0)
      / NULLIF(sol.vl_emprestimo_total_ano, 0)) * 100,
    3
  ) AS pct_perda_sobre_emprestimo

FROM sol
LEFT JOIN ris
  ON sol.ano = ris.ano
 AND sol.sk_localizacao = ris.sk_localizacao
LEFT JOIN puc.gold.dim_localizacao dl
  ON sol.sk_localizacao = dl.sk_localizacao
ORDER BY sol.ano, dl.regiao, dl.uf;


In [0]:
-- Percentual entre o total da perda esperada e o total de empréstimos usando granularidade de trimestre e região

WITH sol AS (
  SELECT
    dt.trimestre,
    f.sk_localizacao,
    SUM(f.valor_emprestimo) AS vl_emprestimo_total_ano
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo dt
    ON f.dt_solicitacao = dt.dt_solicitacao
  GROUP BY dt.trimestre, f.sk_localizacao
),
ris AS (
  SELECT
    dt.trimestre,
    fr.sk_localizacao,
    SUM(fr.perda_esperada_soma) AS perda_esperada_total_ano
  FROM puc.gold.fato_risco_cliente_mes fr
  JOIN puc.gold.dim_tempo dt
    ON fr.dt_mes = dt.dt_solicitacao
  GROUP BY dt.trimestre, fr.sk_localizacao
)
SELECT
  sol.trimestre,
  dl.uf,
  dl.regiao,
  sol.vl_emprestimo_total_ano,
  COALESCE(ris.perda_esperada_total_ano, 0) AS perda_esperada_total_ano,

  -- percentual da perda sobre o empréstimo (em %)
  ROUND(
    (COALESCE(ris.perda_esperada_total_ano, 0)
      / NULLIF(sol.vl_emprestimo_total_ano, 0)) * 100,
    3
  ) AS pct_perda_sobre_emprestimo

FROM sol
LEFT JOIN ris
  ON sol.trimestre = ris.trimestre
 AND sol.sk_localizacao = ris.sk_localizacao
LEFT JOIN puc.gold.dim_localizacao dl
  ON sol.sk_localizacao = dl.sk_localizacao
ORDER BY sol.trimestre, dl.regiao, dl.uf;


In [0]:
-- Percentual entre o total da perda esperada e o total de empréstimos
-- usando granularidade de ANO + TRIMESTRE + (UF/Região)

WITH sol AS (
  SELECT
    dt.ano,
    dt.trimestre,
    f.sk_localizacao,
    SUM(f.valor_emprestimo) AS vl_emprestimo_total
  FROM puc.gold.fato_solicitacao_credito f
  JOIN puc.gold.dim_tempo dt
    ON f.dt_solicitacao = dt.dt_solicitacao
  GROUP BY dt.ano, dt.trimestre, f.sk_localizacao
),
ris AS (
  SELECT
    dt.ano,
    dt.trimestre,
    fr.sk_localizacao,
    SUM(fr.perda_esperada_soma) AS perda_esperada_total
  FROM puc.gold.fato_risco_cliente_mes fr
  JOIN puc.gold.dim_tempo dt
    ON fr.dt_mes = dt.dt_solicitacao
  GROUP BY dt.ano, dt.trimestre, fr.sk_localizacao
)
SELECT
  sol.ano,
  sol.trimestre,
  dl.uf,
  dl.regiao,
  sol.vl_emprestimo_total,
  COALESCE(ris.perda_esperada_total, 0) AS perda_esperada_total,

  ROUND(
    (COALESCE(ris.perda_esperada_total, 0)
      / NULLIF(sol.vl_emprestimo_total, 0)) * 100,
    3
  ) AS pct_perda_sobre_emprestimo

FROM sol
LEFT JOIN ris
  ON sol.ano = ris.ano
 AND sol.trimestre = ris.trimestre
 AND sol.sk_localizacao = ris.sk_localizacao
LEFT JOIN puc.gold.dim_localizacao dl
  ON sol.sk_localizacao = dl.sk_localizacao
ORDER BY sol.ano, sol.trimestre, dl.regiao, dl.uf;
